In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

Using TensorFlow backend.


In [18]:
# need to get data setup and organized (it is mostly)
data = []
with open('TEST') as file:
    for line in file:
        data.append([float(x) for x in line.split()[1:]])
print(len(data))
print(len(data[0]))

newArray = np.array(data)
output_shape = newArray.shape[0]
input_shape = newArray.shape[1]

100
50


In [31]:
model = keras.Sequential()

model.add(keras.layers.Conv1D(50, kernel_size=(5), strides=(1),
                 activation='relu',
                 input_shape=[newArray.shape[0],
                              newArray.shape[1]]))

model.add(keras.layers.SimpleRNN(100, activation='tanh', use_bias=True, 
                       kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                       bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, 
                       bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
                       recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, 
                       return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False))

model.add(keras.layers.Dropout(.20, noise_shape=None, seed=None))
#model.add(keras.layers.MaxPooling1D(pool_size=(2), strides=(2)))
#model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1000, activation='relu'))
model.add(keras.layers.Dense(100, activation='softmax'))


## Citations